Idea:

- Each number is independent. 
- Hence, the total repetitions for each num will likely be equal in the long term.
- Find the relations between a number to the rest within each draw. 
  - For example, if number 1 is drawn, what are the chances for the other particular number to be drawn based on the previous draws.
  - So each number will have a dictionary of other 44 numbers while each nested dictionary has two keys: Winning and Supps.
Later, give more weights to those have higher chances when generating a random ticket number.


The process:
1. Scan the previous draws and list the dictionary for the number relations.
2. Setting some threshold and weights to generate a random ticket
3. Each play has 10 tickets
4. Check the results: Hypothesis tests


In [22]:
# Library Imports
import csv
import random
import pandas as pd
import time
from itertools import islice
from enum import Enum

In [2]:
# Previous Lotto Records into Lists
wedLotto = []
satLotto = []
with open('Wednesday Lotto Results - G NETWORK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        wedLotto.append(row)

with open('Saturday Lotto Results - G NETWORK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        satLotto.append(row)
        
wedLotto = wedLotto[1::]
satLotto = satLotto[1::]

In [3]:
# Go through the previous draws and mark the relations between numbers
wedRelations = {}
satRelations = {}

# list, set, dict are mutable -> pass by reference (if not reassign)
# ig. relationSetter(wedRelations, wedLotto)
def relationSetter(relations, prev_draws) -> dict:
    # String List to Int List
    for draw in prev_draws:
        try:
            drawnTicket = list(map(int, draw[1::]))
        except:
            continue
        for myIndex in range (0, len(drawnTicket)):
            num = drawnTicket[myIndex]
            for itsIndex in range (0, len(drawnTicket)):
            #for num in drawnTicket:
                itsSupp = True if itsIndex > 5 else False
                numNum = drawnTicket[itsIndex]
                if num != numNum: # Not repeating myself
                    if not num in relations:
                        relations[num] = {}
                    if not numNum in relations[num]:
                        relations[num][numNum] = 0
                    else:
                        relations[num][numNum] += 1

In [4]:
# Get relation between the numbers
relationSetter(wedRelations, wedLotto)
relationSetter(satRelations, satLotto)

In [5]:
df_wed = pd.DataFrame.from_dict(wedRelations)
df_sat = pd.DataFrame.from_dict(satRelations)

In [6]:
def thresholdCalc(rep, mean) -> float:
    # if rep > mean -> positive
    # else negative
    return ((rep/mean) - 1)

# Get best pals (numbers) to be for a particular number
# Currently set min num of pals = 5
min_num_pals = 5
def closePalsGenerator(tableX, asked_num_pals) -> dict:
    sorted_table = tableX.sort_values(ascending= False)
    retDict = {}
    table_mean = sorted_table.mean()
    pals_sum = 0
    for key, value in sorted_table[:asked_num_pals].items():
        retDict[key] = value
        pals_sum += value

    retDict['all_mean'] = round(table_mean, 2)
    retDict['pals_mean'] = pals_sum / asked_num_pals
    # chosen pals have <> higher chance to be selected.
    retDict['perc'] = round ((retDict['pals_mean'] / retDict['all_mean'])*100 - 100, 2)
    return retDict
    
# df_<>_closePals : include num_close_pals with its overall mean val
df_wed_closePals = {}
df_sat_closePals = {}
for num in range (1,46):
    df_wed_closePals[num] = closePalsGenerator(df_wed.get(num), min_num_pals)

for num in range (1,46):
    df_sat_closePals[num] = closePalsGenerator(df_sat.get(num), min_num_pals)
   

In [101]:
# Generate combinations
# 1. randomly generate a starting number
# 2. Choose the next number with threshold
# threshold: percentage level?
# 3. If threshold low -> random?
# ticketGenerator(df_sat_closePals, anyNum)
def ticketGenerator(closePals, threshold) -> dict:
    retDict = {}
    one_ticket_length = 8
    # Generate one number randomly + Default Setup
    # - Remember numbers are evenly distributed in the long term
    last_num = random.sample(range(1,46), 1)[0]
    retDict['game'] = [last_num]
    # perc_mean -> indicate chances...
    perc_sum = closePals.get(last_num)['perc']
    # Generate ticket
    guard_count = 0
    while (len(retDict['game']) < one_ticket_length):
        chosen = random.choice(list(closePals.get(last_num).keys()))
        while type(chosen) == str or chosen in retDict['game']:
            if guard_count > 100:
                chosen = random.sample(range(1,46), 1)[0]
                while chosen in retDict['game']:
                    chosen = random.sample(range(1,46), 1)[0]
                continue
                
            chosen = random.choice(list(closePals.get(last_num).keys()))
            guard_count += 1
        # now chosen is a (num, val) that is not part of retDict['ticket']
        retDict['game'].append(chosen)
        perc_sum += closePals.get(chosen)['perc']
        last_num = chosen
        
        
    
    # To return
    retDict['perc_mean'] = round(perc_sum / len(retDict['game']),2)
    return retDict

# enum for diff lotto
class Lotto(Enum):
    WED = 1
    SAT = 2
    
class Division(Enum):
    first = "First"
    second = "Second"
    third = "Third"
    fourth = "Fourth"
    fifth = "Fifth"
    sixth = "Sixth"
    failed = "Failed"
    
def randomGenerator() -> list:
    games = []
    for i in range (0,10):
        games.append(sorted(random.sample(range(1,46), 8)))
        
    return games

def gamesGenerator() -> list:
    games = []
    for i in range(0,1000):
        games.append(ticketGenerator(df_wed_closePals, 0.6))
    # Sort the 1000 combinations, and choose 10 best perc games
    sorted_games = sorted(games, key = lambda game: game['perc_mean'], reverse=True)
    glorious_games = []
    for game in sorted_games[:10]:
        glorious_games.append(game['game'])
    
    return glorious_games   

def divisionCalc(game, lotto ,drawn) -> str:
    winning_match = 0
    supp_match = 0
    # To compare two lists -> need two indices
    drawnIndex = 0
    gameIndex = 0
    for game_num in game:
        indices = [index for index, val in enumerate(drawn) if game_num == val]
        if indices != []:
            if indices[0] > 5:
                supp_match += 1
            else:
                winning_match += 1
    # Division Check
    # Rules - Common
    # 1. 6 winnings 2. 5 winnings + 1 supp 3. 5 winnings
    # 4. 4 winnings 
    if winning_match == 6:
        return Division.first.value
    elif winning_match == 5:
        if supp_match == 1:
            return Division.second.value
        else:
            return Division.third.value
    elif winning_match == 4:
        return  Division.fourth.value
    
    # Rules - Wed
    # 5. 3 winnings + 2 supp 6. total 3 (winning 1 + 2, 2 + 1)
    # Rules - Sat
    # 5. 3 winnings + 1 supp 6. 3 winnings
    if lotto == Lotto.WED:
        if winning_match == 3 and supp_match == 2:
            return Division.fifth.value
        else:
            if winning_match + supp_match == 3:
                return Division.sixth.value
            else:
                return Division.failed.value
    else:
        if winning_match == 3 and supp_match == 2:
            return Division.fifth.value
        else:
            if winning_match == 3:
                return Division.sixth.value
            else:
                return Division.failed.value
            
# games have 10 games, lotto = Lotto.enum, drawnNums = Draws
def gamesMarker(games, lotto, drawnNums, results) -> dict:
    winning_match = 0
    supp_match = 0
    for game in games:
        results[divisionCalc(game, lotto, drawnNums)] += 1

def autoTester(f, lotto_data, lotto_enum):
    final_output = {'First':0, 'Second':0, 'Third':0, 'Fourth':0,
              'Fifth':0, 'Sixth':0, 'Failed':0}
    for draw in lotto_data:
        games = f()
        gamesMarker(games, lotto_enum, draw[1::], final_output)
        #print(result)
    print(final_output)

In [102]:
#gamesGenerator()
randomGenerator()

[[2, 13, 20, 21, 22, 31, 41, 44],
 [1, 5, 10, 26, 28, 35, 38, 45],
 [6, 8, 10, 16, 20, 23, 28, 36],
 [2, 5, 7, 11, 12, 18, 29, 34],
 [4, 6, 14, 16, 22, 28, 36, 43],
 [8, 14, 22, 29, 36, 38, 41, 44],
 [6, 15, 16, 18, 29, 35, 36, 39],
 [1, 8, 15, 19, 24, 30, 40, 44],
 [10, 11, 12, 20, 22, 26, 31, 41],
 [5, 11, 21, 22, 26, 29, 34, 37]]

{'First': 0, 'Second': 0, 'Third': 0, 'Fourth': 0, 'Fifth': 0, 'Sixth': 0, 'Failed': 5600}
Took Time:  15 2


In [106]:
# Want to see the winning rate over the period
start_time = time.time()
#autoTester(gamesGenerator, wedLotto, Lotto.WED)
autoTester(randomGenerator, wedLotto, Lotto.WED)
print("Took Time: ", (time.time() - start_time))

{'First': 0, 'Second': 0, 'Third': 0, 'Fourth': 0, 'Fifth': 0, 'Sixth': 0, 'Failed': 5600}
Took Time:  0.10007882118225098
